### Importing Libraries

In [1]:
import os
import cv2
import numpy as np
import torch
import zipfile
from torchvision.transforms import transforms
import tensorflow as tf
from torch.utils.data import random_split, DataLoader, TensorDataset
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = [5, 5]

In [2]:
with zipfile.ZipFile("al_data.zip", "r") as zip_file:
  zip_file. extractall('')

### Load Dataset

In [3]:
train_path = 'al_data/train'
test_path = 'al_data/test'

In [4]:
def read_data(data_path):
    dataset = []
    labels = []
    DIRPATH = list(os.listdir(data_path))
    for i in range(len(DIRPATH)):
        for img in os.listdir(os.path.join(data_path, DIRPATH[i])):
            image = cv2.imread(os.path.join(data_path, DIRPATH[i], img))//255
            labels.append(i)
            dataset.append(image)
    return dataset, labels

In [5]:
def create_data(data, lables):
    # dataset = []
    # for i in range(len(data)):
    #     dataset.append([data[i], labels[i]])
    dataset = torch.cat((data, lables.unsqueeze(1)), dim=1)
    return dataset

In [6]:
tr_data, tr_lables = read_data(train_path)
te_data, te_lables = read_data(test_path)
len(tr_data), len(te_data)

(5121, 1279)

In [7]:
train_array = np.array(tr_data, dtype=np.float32)
train_dataset = torch.tensor(train_array)

In [8]:
train_lables = torch.tensor(tr_lables)

In [9]:
test_array = np.array(te_data, dtype=np.float32)
test_dataset = torch.from_numpy(test_array)

In [12]:
test_lables = torch.tensor(te_lables)

In [13]:
type(train_dataset), type(train_lables), type(test_dataset), type(test_lables)

(torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor)

In [13]:
train_lables[:5]

tensor([0, 0, 0, 0, 0])

In [14]:
training_datasets = TensorDataset(train_dataset, train_lables)
testing_datasets = TensorDataset(test_dataset, test_lables)

In [15]:
train_dataset, dev_dataset = training_datasets[int(len(training_datasets)*0.8):], training_datasets[:int(len(training_datasets)*0.8):]

In [17]:
print(len(train_dataset[1]))

1025


In [ ]:
print(len(train_datasets[0]), len(test_datasets[0]))

In [ ]:
dev_len = len(train)//15
dev_dataset = train[:dev_len]
len(train), len(dev_dataset)

In [ ]:
print(len(train[1]), len(test[1]))

### Define Notebook Constants

In [ ]:
total_train_size = len(train)
total_test_size = len(test)
total_dev_size = len(dev_dataset)

classes = 10
input_dim = 784

num_clients = 4
rounds = 10
batch_size = 128
epochs_per_client = 3
learning_rate = 2e-2

In [ ]:
total_train_size, total_dev_size, total_test_size

### Define utilities for GPU support

In [ ]:
def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader(DataLoader):
        def __init__(self, dl, device):
            self.dl = dl
            self.device = device

        def __iter__(self):
            for batch in self.dl:
                yield to_device(batch, self.device)

        def __len__(self):
            return len(self.dl)

device = get_device()

### Define FederatedNet class

In [ ]:
class FederatedNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 20, 7)
        self.conv2 = torch.nn.Conv2d(20, 40, 7)
        self.maxpool = torch.nn.MaxPool2d(2, 2)
        self.flatten = torch.nn.Flatten()
        self.linear = torch.nn.Linear(2560, 10)
        self.non_linearity = torch.nn.functional.relu
        self.track_layers = {'conv1': self.conv1, 'conv2': self.conv2, 'linear': self.linear}

    def forward(self, x_batch):
        out = self.conv1(x_batch)
        out = self.non_linearity(out)
        out = self.conv2(out)
        out = self.non_linearity(out)
        out = self.maxpool(out)
        out = self.flatten(out)
        out = self.linear(out)
        return out

    def get_track_layers(self):
        return self.track_layers

    def apply_parameters(self, parameters_dict):
        with torch.no_grad():
            for layer_name in parameters_dict:
                self.track_layers[layer_name].weight.data *= 0
                self.track_layers[layer_name].bias.data *= 0
                self.track_layers[layer_name].weight.data += parameters_dict[layer_name]['weight']
                self.track_layers[layer_name].bias.data += parameters_dict[layer_name]['bias']

    def get_parameters(self):
        parameters_dict = dict()
        for layer_name in self.track_layers:
            parameters_dict[layer_name] = {
                'weight': self.track_layers[layer_name].weight.data,
                'bias': self.track_layers[layer_name].bias.data
            }
        return parameters_dict

    def batch_accuracy(self, outputs, labels):
        with torch.no_grad():
            _, predictions = torch.max(outputs, dim=1)
            return torch.tensor(torch.sum(predictions == labels).item() / len(predictions))

    def _process_batch(self, batch):
        images, labels = batch
        outputs = self(images)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        accuracy = self.batch_accuracy(outputs, labels)
        return (loss, accuracy)

    def fit(self, dataset, epochs, lr, batch_size=128, opt=torch.optim.SGD):
        dataloader = DeviceDataLoader(DataLoader(dataset, batch_size, shuffle=True), device)
        optimizer = opt(self.parameters(), lr)
        history = []
        for epoch in range(epochs):
            losses = []
            accs = []
            for batch in dataloader:
                loss, acc = self._process_batch(batch)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                loss.detach()
                losses.append(loss)
                accs.append(acc)
            avg_loss = torch.stack(losses).mean().item()
            avg_acc = torch.stack(accs).mean().item()
            history.append((avg_loss, avg_acc))
        return history

    def evaluate(self, dataset, batch_size=128):
        dataloader = DeviceDataLoader(DataLoader(dataset, batch_size), device)
        losses = []
        accs = []
        with torch.no_grad():
            for batch in dataloader:
                loss, acc = self._process_batch(batch)
                losses.append(loss)
                accs.append(acc)
        avg_loss = torch.stack(losses).mean().item()
        avg_acc = torch.stack(accs).mean().item()
        return (avg_loss, avg_acc)

### Define Client class

In [ ]:
class Client:
    def __init__(self, client_id, dataset):
        self.client_id = client_id
        self.dataset = dataset

    def get_dataset_size(self):
        return len(self.dataset)

    def get_client_id(self):
        return self.client_id

    def train(self, parameters_dict, return_model_dict=False):
        net = to_device(FederatedNet(), device)
        net.apply_parameters(parameters_dict)
        train_history = net.fit(self.dataset, epochs_per_client, learning_rate, batch_size)
        print(self.client_id + ':')
        for i, res in enumerate(train_history):
            print('Epoch [{}]: Loss = {}, Accuracy = {}'.format(i + 1, round(res[0], 4), round(res[1], 4)))
        return net.get_parameters(), net.state_dict() if return_model_dict else None

### Setup clients

In [ ]:
examples_per_client = total_train_size // num_clients
client_datasets = random_split(train, [min(i + examples_per_client,
           total_train_size) - i for i in range(0, total_train_size, examples_per_client)])
clients = [Client('client_' + str(i), client_datasets[i]) for i in range(num_clients)]

In [ ]:
print(clients)

### Start server

In [ ]:
global_net = to_device(FederatedNet(), device)
history = []
for i in range(rounds):
    print('Start Round {} ...'.format(i + 1))
    curr_parameters = global_net.get_parameters()
    new_parameters = dict([(layer_name, {'weight': 0, 'bias': 0}) for layer_name in curr_parameters])
    for client in clients:
        client_parameters = client.train(curr_parameters)
        fraction = client.get_dataset_size() / total_train_size
        for layer_name in client_parameters:
            new_parameters[layer_name]['weight'] += fraction * client_parameters[layer_name]['weight']
            new_parameters[layer_name]['bias'] += fraction * client_parameters[layer_name]['bias']
    global_net.apply_parameters(new_parameters)

    train_loss, train_acc = global_net.evaluate(training_data)
    dev_loss, dev_acc = global_net.evaluate(dev_dataset)
    print('After round {}, train_loss = {}, dev_loss = {}, dev_acc = {}\n'.format(i + 1, round(train_loss, 4),
            round(dev_loss, 4), round(dev_acc, 4)))
    history.append((train_loss, dev_loss))

In [ ]:
plt.plot([i + 1 for i in range(len(history))], [history[i][0] for i in range(len(history))], color='r', label='train loss')
plt.plot([i + 1 for i in range(len(history))], [history[i][1] for i in range(len(history))], color='b', label='dev loss')
plt.legend()
plt.title('Training history')
plt.show()